In [3]:
# Cell 1: Imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [4]:
# Cell 2: Set device for RTX 3090
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Cell 3: Load MNIST dataset
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:00<00:00, 25.5MB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 932kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 7.94MB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<?, ?B/s]


In [6]:
# Cell 4: Define neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
# Cell 5: Set loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
# Cell 6: Training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
# Cell 7: Testing function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
# Cell 8: Train and test for 5 epochs
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.525807  [    0/60000]
loss: 0.285705  [ 6400/60000]
loss: 0.293069  [12800/60000]
loss: 0.219062  [19200/60000]
loss: 0.111555  [25600/60000]
loss: 0.108816  [32000/60000]
loss: 0.067624  [38400/60000]
loss: 0.094229  [44800/60000]
loss: 0.094860  [51200/60000]
loss: 0.054134  [57600/60000]
Test Error: 
 Accuracy: 96.9%, Avg loss: 0.096183 

Epoch 2
-------------------------------
loss: 0.116420  [    0/60000]
loss: 0.041614  [ 6400/60000]
loss: 0.010022  [12800/60000]
loss: 0.045220  [19200/60000]
loss: 0.106489  [25600/60000]
loss: 0.042193  [32000/60000]
loss: 0.179749  [38400/60000]
loss: 0.025186  [44800/60000]
loss: 0.060889  [51200/60000]
loss: 0.051480  [57600/60000]
Test Error: 
 Accuracy: 97.3%, Avg loss: 0.085816 

Epoch 3
-------------------------------
loss: 0.086285  [    0/60000]
loss: 0.067953  [ 6400/60000]
loss: 0.018865  [12800/60000]
loss: 0.122583  [19200/60000]
loss: 0.005598  [25600/60000]
loss: 0.059390  [32000/600

In [11]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3090
